In [ ]:
!pip install -U torch  transformers sentence-transformers keybert pytextrank
!python -m spacy download en_core_web_sm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install spacy-transformers

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from nltk import sent_tokenize
import torch
import numpy as np
from scipy.signal import argrelextrema
import spacy
import pytextrank
from keybert import KeyBERT
from nltk.corpus import stopwords
from pprint import pprint

In [ ]:
HF_MODEL = 'paraphrase-MiniLM-L6-v2'
sentence_encoder_model = SentenceTransformer(HF_MODEL)
kw_model = KeyBERT(model=HF_MODEL)
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("textrank")


In [ ]:
def split_list(a, n):
    k, m = divmod(len(a), n)
    return (
        a[i * k + min(i, m) : (i + 1) * k + min(i + 1, m)]
        for i in range(min(len(a), n))
    )


def sentences_similarity(first_sentence_features, second_sentence_features) -> float:
    similarity_metric = torch.nn.CosineSimilarity()
    return float(similarity_metric(first_sentence_features, second_sentence_features))

def compute_window(timeseries, start_index, end_index):
    features = timeseries[start_index:end_index].unsqueeze(0)
    stack_size = end_index - start_index
    pooling = torch.nn.MaxPool2d((stack_size - 1, 1))
    return pooling(features)

def block_comparison_score(features, window_size=14):
    res = []
    for i in range(window_size, len(features) - window_size):
        first_window_features = compute_window(features, i - window_size, i + 1)
        second_window_features = compute_window(features, i + 1, i + window_size + 2)
        res.append(
            sentences_similarity(first_window_features[0], second_window_features[0])
        )

    return res

def smooth(features, num_passes, window):
    smoothed_features = features[:]
    for _ in range(num_passes):
        for index in range(len(smoothed_features)):
            neighbours = smoothed_features[
                max(0, index - window) : min(len(features) - 1, index + window)
            ]
            smoothed_features[index] = sum(neighbours) / len(neighbours)
    return smoothed_features

def depth_score(features):
    depth_scores = []
    for i in range(1, len(features) - 1):
        left, right = i - 1, i + 1
        while left > 0 and features[left - 1] > features[left]:
            left -= 1
        while (
            right < (len(features) - 1) and features[right + 1] > features[right]
        ):
            right += 1
        depth_scores.append(
            (features[right] - features[i]) + (features[left] - features[i])
        )
    return depth_scores

def get_local_maxima(array):
    if isinstance(array, list):
      array = np.array(array)
    local_maxima_indices = argrelextrema(array, np.greater)[0].tolist()
    return local_maxima_indices, array[local_maxima_indices].tolist()


def arsort2(array1, array2):
    x = np.array(array1)
    y = np.array(array2)

    sorted_idx = x.argsort()[::-1]
    return x[sorted_idx], y[sorted_idx]


def depth_score_to_topic_change_indexes(
    depth_score_timeseries,
    meeting_duration,
    capped=True,
    average_segment_length=50, 
    max_segments=10,
    threshold_fraction=0.5):

    if len(depth_score_timeseries) == 0:
        return []
    threshold = threshold_fraction * max(depth_score_timeseries)

    
    local_maxima_indices, local_maxima = get_local_maxima(depth_score_timeseries)

    if len(local_maxima) == 0:
        return []
    if capped:  
        # sort based on maxima for pruning
        local_maxima, local_maxima_indices = arsort2(local_maxima, local_maxima_indices)

        # local maxima are sorted by depth_score value and we take only the first K
        # where the K+1th local maxima is lower then the threshold
        for thres in range(len(local_maxima)):
            if local_maxima[thres] <= threshold:
                break

        max_segments = min(int(meeting_duration / average_segment_length), max_segments)
        slice_length = min(max_segments, thres)

        local_maxima_indices = local_maxima_indices[:slice_length]
        local_maxima = local_maxima[:slice_length]

        # after pruning, sort again based on indices for chronological ordering
        local_maxima_indices, _ = arsort2(local_maxima_indices, local_maxima)

    else:  # this is the vanilla TextTiling used for Pk optimization
        filtered_local_maxima_indices = []
        filtered_local_maxima = []

        for i, m in enumerate(local_maxima):
            if m > threshold:
                filtered_local_maxima.append(m)
                filtered_local_maxima_indices.append(i)

        local_maxima = filtered_local_maxima
        local_maxima_indices = filtered_local_maxima_indices

    return local_maxima_indices


def segment_meeting(
    df, utterance_col="utterance",
    batch_size=10, 
    sentence_comparision_window=15,
    num_passes=2,
    gap_window=1,
    capped=True,
    average_segment_length=50, 
    max_segments=10,
    threshold_fraction=0.5):
  
  batches_features = []
  utterances = df[utterance_col].tolist()
  for batch_start in range(0,len(df), batch_size):
    batch_sentences = utterances[batch_start:batch_start+batch_size]
    batches_features.append(sentence_encoder_model.encode(batch_sentences))
  features = torch.FloatTensor([feature for features in batches_features for feature in features])
  gap_scores = block_comparison_score(features, window_size=sentence_comparision_window)

  # chang this 
  gap_scores = smooth(gap_scores, num_passes=num_passes, window=gap_window,)
  print(len(gap_scores))

  depth_score_features = depth_score(gap_scores)
  print(len(depth_score_features))

  segment_idxs = depth_score_to_topic_change_indexes(
      depth_score_features,len(meeting_df),capped=capped,
      average_segment_length=average_segment_length, 
    max_segments=max_segments,
    threshold_fraction=threshold_fraction)
  segment_idxs = [0] + sorted(segment_idxs) + [len(df)]
  list_of_segments_df = [meeting_df.iloc[segment_idxs[n]:segment_idxs[n+1]] for n in range(len(segment_idxs)-1)]
  return list_of_segments_df

In [ ]:
from spacy.tokens import Doc

def extract_candidate_topics(docs):
  doc = Doc.from_docs(docs)
  text = "\n".join(sent.text for sent in doc.sents if len(sent)> 3)
  # examine the top-ranked phrases in the document
  chunks = list(map(lambda x: x.text, doc.ents))
  chunks += list(map(lambda x: x.text, doc.noun_chunks))
  for p in doc._.phrases:
      # print('{:.4f} {:5d}  {}'.format(p.rank, p.count, p.text))
      # print(p.chunks)
      chunks.append(p.chunks[0].text)

  candidates_scores = kw_model.extract_keywords(text,
      stop_words='english', 
      use_mmr=True, diversity=0.9)
  candidates, _scores = zip(*candidates_scores)

  candidates_scores = kw_model.extract_keywords(
      text,keyphrase_ngram_range=(2, 2),
      stop_words='english', 
      use_mmr=True, diversity=0.9)
  candidates2, _scores = zip(*candidates_scores)
  
  # candidates_scores = kw_model.extract_keywords(
  #     text,keyphrase_ngram_range=(3, 3),
  #     stop_words='english', 
  #     use_maxsum=True, diversity=0.7)
  # candidates3, _scores = zip(*candidates_scores)
  candidates_list = chunks+list(candidates)+list(candidates2) # +list(candidates3)
  candidates = kw_model.extract_keywords(
      text, candidates=candidates_list,
      stop_words="english", use_mmr=True, diversity=0.7, top_n=5)
  return candidates


In [ ]:
transcript = [{"start":4.24722,"end":68.6,"utterance":"You've done an amazing job in this book, so trying to boil it dollar for zones like me. Click hard it is it is a complicated and insanely nuance subject. Yeah. And it's it's one of those things where it like this many worlds series to for one example. The the the just the possibility that there's like, explain that. Just explain for for people that don't understand what quantum mechanics even means. Give them just like a little bit of that and then wayne many worlds do. Yeah. Good. This is what I'm here to do. So you know, and take it look like. Plum mechanics should apply to the entire universe but it becomes unmistakable when you look at little tiny. Right? So we always are talking about electrons or atoms and so forth. An electron has a position and Well, sorry. Let me not even say that. Even that was wrong. It's just so hard to correctly talk about tomorrow i mechanics. Right? If you were isaac newton... Before there's quantum mechanic think, there's classical mechanics. And basically, one we can make some custom mechanics with the only two big crank"},{"start":71.21417,"end":100.20258,"utterance":"was so good that everyone on that was right and all better building the details until on mechanics came along and changed it. In classical mechanics and electron is a point. It has a position the location and space and has velocity moving somewhere. And from that, you can predict what's gonna happen. Okay? Quantum mechanics says, no. No. No. The electrons has a wave function. So if there's a way you know, sometimes you hear this debate developer or things like electrons and photons articles or waiting. The answer is that they're weights."},{"start":101.78023,"end":161.43192,"utterance":"And the way function has this weird property that when you're not looking at it, it's a way. It's all spread out or it's localized somewhere, but it'll obey an equation shorten your equation. So far i'm good just the regular physics. There's a thing. Wave function basic and equation and for equation predict what's gonna have next. But the weird thing about one we mechanics is that there's a whole separate set of rules for what happens when you look at the things, when you observe it, when you measure it, that's where things get score with people describing Right? Yeah. And that's where they wanna go Google on. It's an opening to be moved. Right? We say, like, what do you mean observe something? Like, so that be a account just being, could be the deal camera, you know, that just brand years. Right. Is it the act of measuring the changes thing? Well, this is the puzzle. Okay. This is what is called the measurement problem of quantum mechanics, so that the rules we teach in our students at Contact or anywhere else we some quantum mechanics and their software your knowledge."},{"start":162.64656,"end":182.21042,"utterance":"The rule say when a system is observed when it is measured, it stays its way function changes dramatically only. Now let me ask you that's how do we know this based on... If if you're measuring it and it changes, how do we know because we didn't measure it before. Like,"},{"start":184.61758,"end":221.41086,"utterance":"that we understand the state of it before it's measured, but without measuring it. Good just a couple of ways. So Let let me make things even simpler forget about where the fund is located and think about the electronic spinning. Right? Like on spending just like the it's really exactly like that. Like whole spending company accept. When you measure this thing, you can sort of send the electron through magnetic field and it would get this like either up or down depending on whether it's spending been out or been down. You only ever get one or two answers it's either going up for going in between an empirical measure. I just recording a meeting. Hey. Hi. Can you hear me?"},{"start":224.37755,"end":233.71277,"utterance":"Yeah. I was Yeah. Was recording a meeting. Podcast cars, I was running it. So three minutes fifty two seconds, I think ten minutes would be the Id one. Right?"},{"start":237.61572,"end":238.61273,"utterance":"Oh yeah. Okay."},{"start":248.55707,"end":248.63681,"utterance":"Okay."},{"start":281.9347,"end":282.01462,"utterance":"Okay."},{"start":302.8814,"end":302.9613,"utterance":"Okay."},{"start":311.63675,"end":311.75647,"utterance":"Yeah."},{"start":316.0463,"end":316.60574,"utterance":"Right. Yeah."},{"start":320.80148,"end":321.04126,"utterance":"Mh."},{"start":325.24817,"end":325.4078,"utterance":"Yeah."},{"start":344.51498,"end":344.87497,"utterance":"For today?"},{"start":348.96997,"end":349.87,"utterance":"Okay. Then"},{"start":351.81,"end":362.78058,"utterance":"can we talk to Adi can say that can you say that I only leave today? Because I got... I get stuck some calls, you know, and those waste a lot of time to be really honest."},{"start":365.40588,"end":366.16388,"utterance":"Call... Yeah."},{"start":371.88495,"end":375.82495,"utterance":"Yeah. I mean, call if anything in. Yeah. Okay"},{"start":418.685,"end":419.845,"utterance":"i dropped on my system."},{"start":426.285,"end":426.445,"utterance":"Okay."},{"start":428.01807,"end":457.33374,"utterance":"All of these things. Right? Running transcriptions. Okay. That the way we they are done they can modify it a little bit so that can be quickly. Segment... Yeah. So that needs to be seen because now we are not active working with Sig. We be putting just like the moment time then, we'll be putting some days and all we on click will apply in that we have to check the, you know, the position"},{"start":458.3878,"end":459.44565,"utterance":"where the click happened."},{"start":464.3436,"end":471.24216,"utterance":"Yeah. And moments. Okay. So I have captured a few let me get one. Three four months?"},{"start":472.2571,"end":485.11002,"utterance":"While we are at. I just want to have some transcription which is running for eight minutes now. So... Yeah. Second one. So if I have a Lengthy recording and, you know, one or two two or three captures."},{"start":487.30396,"end":501.51,"utterance":"And hopefully, I get those details in the then... Yeah. I think we should be able to do it. Let me see. I need to... Yeah. Press it today. But then yeah, again there is no distraction of any. Sort. So I'll just talk to Adi. It's not... Okay."},{"start":511.92938,"end":515.157,"utterance":"So today, and I won't call with a. So you don't call me an authenticate."},{"start":517.44507,"end":527.44006,"utterance":"Can. Yeah. So you have to do it before level thirty something. Then because the client calls, let me see what most of times."},{"start":529.94006,"end":530.44006,"utterance":"The"},{"start":532.42957,"end":561.9037,"utterance":"We daily have sync of eight digital one with the you know, the scrum, and one with this decide who is going to be lead for end. So he'll be assigning me work he'll be giving me the report details and everything. So the discussion initial level discussion will happen delete. And They also have joined the flag. With him. Now Nicola, I don't know. So I don't know why i she's out of the picture, but seems like, I am the only one who's looking at different front and from politicians."},{"start":564.31244,"end":568.1388,"utterance":"Yeah. This have there. Okay. Yeah this morning."},{"start":574.4965,"end":574.61633,"utterance":"Yeah."},{"start":629.20496,"end":632.40497,"utterance":"Yeah. This... I think in the side. Yeah."},{"start":656.0729,"end":656.3522,"utterance":"Yeah."},{"start":657.60504,"end":676.74426,"utterance":"Problem with Adi. So sai, we need to connect... So one is scrum call is I need to sync up this. I he will you know, message slack, and we'll have to join a meeting. We have to record the session. We'll he'll give me all the details. So those sessions also might they take up a lot of bandwidth without time for minute."},{"start":683.20996,"end":692.4428,"utterance":"Yeah. So today, and this week they are planning to share the report and every other details and he's expecting us to start working. By the end of the week itself. So... Yeah."},{"start":699.91,"end":700.15,"utterance":"Yeah."},{"start":704.58997,"end":704.63,"utterance":"Okay."},{"start":719.1963,"end":721.9838,"utterance":"Okay. Thank you, Adi. Yeah. Right."},{"start":724.2377,"end":724.3574,"utterance":"Yeah."},{"start":731.8352,"end":732.0347,"utterance":"Yeah."},{"start":736.1443,"end":736.3039,"utterance":"Yeah."},{"start":739.5496,"end":739.749,"utterance":"Yes."},{"start":745.05225,"end":746.3098,"utterance":"Alright. Okay. So"},{"start":751.38,"end":763.595,"utterance":"Okay. Right. So next thing what I'll do is that will be set forward me at least at least under the transcription. Not the running ones but at least the text. Right? That's pretty doable. It's very easy. After the player, not... Yeah."},{"start":768.92993,"end":771.70996,"utterance":"Moment cards. Okay. Highlight ping of."},{"start":779.785,"end":780.345,"utterance":"Yeah. Yeah."},{"start":786.2365,"end":788.31433,"utterance":"Let it move it scroll as well. Okay."},{"start":803.05707,"end":803.13696,"utterance":"Okay."},{"start":813.8555,"end":814.8136,"utterance":"Nine. So yeah."},{"start":816.45026,"end":818.76556,"utterance":"I'm just thinking of this clothing thing."},{"start":824.6071,"end":824.68695,"utterance":"Okay."},{"start":837.70685,"end":841.88293,"utterance":"Okay. Hold. I'll just check to it. So?"},{"start":852.5693,"end":860.732,"utterance":"So I'm just thinking it moment time beginning to if it falls under that then I can"},{"start":861.92413,"end":871.7484,"utterance":"between the beside of paragraph? Yeah. That can be done. That's not an issue. More. I'm just telling about the words the word that I have to automatically"},{"start":892.0856,"end":893.0818,"utterance":"Okay. Sure."},{"start":894.2773,"end":895.4329,"utterance":"Yeah. Okay."},{"start":897.48364,"end":897.76337,"utterance":"Bye."},{"start":904.3165,"end":905.0757,"utterance":"Okay. Cool. Yeah."},{"start":914.265,"end":919.185,"utterance":"Okay. So now I'm not getting about the snapshot for now. Okay. I'm just pushing with no verified flags."},{"start":920.385,"end":922.485,"utterance":"Be before I."},{"start":925.04,"end":928.4,"utterance":"Hello? Yeah. Cool. Okay."},{"start":930.3333,"end":930.8914,"utterance":"Okay. Bye."},{"start":939.64,"end":940.14,"utterance":"Okay."},{"start":941.32,"end":968.4192,"utterance":"So part the quantum mechanics that that's the quantum fact that there's discrete set possible answers to since spinning or counterclockwise? Yes, or no. Just those two possibilities no in between. So if you're you have a magnetic field that is oriented vertically. Send your through and it can just reflected it up and say, oh, it's spin up. So Now I measured it expense. Now I know what it's taken. If I send it through another magnetic"},{"start":969.5174,"end":995.69574,"utterance":"vertically, it was always reflected up every single time. You know what it is. We're gonna measure it measuring it. In this case doesn't change it. It's exactly got safe we know it's Okay. Now let's through magnetic deals that is where get horizontal off. So it's gonna be just like they either right or less. We know exactly what state this way. But when you send it to that that deal the oriented entered horizontal, it kept reflected left or right fifty fifty."},{"start":997.12445,"end":1023.46344,"utterance":"There's no way we can predict it. And then once it is, so you now it's been spinning up. You've measured it's been left, let's say send it to another magnet that is going vertically. And now simply busy again to because get the number sent down. So somehow, even though we do exactly stated it was in, we couldn't predict. What would have the as part of once again. So the act of sending it through these things where makes it vertical or horizontal."},{"start":1024.9174,"end":1165.3767,"utterance":"When what's happening to it when it's going through these things? So in quantum mechanics, what we say is that it's not that we don't know whether the electron has been clockwise or down as loved ones. It can be in a super position about that's just the thin version of the position of the electron can be spread out in a in a way. Right? It's it's truly not just that we are lacking some knowledge that the knowledge really isn't there. And again, this is how we teach quantum mechanics and textbooks. And then I'm gonna correct it, just as many worlds is much better. But this is a standard textbook version. There's a way function, the way function for a spin and see it's either up or down or some combination. And then there's a rule that says when you measure this thing, you only get up or down. You don't see the way function. Just like the cloud that you have for the electron decision, when you look at it, you see it at a location. So another way, if make the same argument is, take a little piece of... I have a nice little image this leg talks, little piece of uranium. So it's a radioactive act little chunk of metal. And you put it in a bubble chamber So it is submitting radioactive particles and you detect the particles. You won't see a little streak of of motion with the park belief. You uranium them okay. Well, like I said, when you're not looking at it, this electron is supposed to obey equation it's a short. And you can ask what the prediction is when is gonna radioactive nucleus sdk and gives off an electrons. What is this wave function than it is. What is the weight function electron is gonna be? The answer is it goes off in a spherical way. It goes off in all directions the ones, even... Yes. All direction evenly. But you never see that. Is that probably based on a shape of the piece of uranium, this is vary? No. Because the electron gets from one individual nucleus of it that. Right? So the... What the... The ringing is doing doesn't matter. Just like one atom of matters. And the easiest thing for the electron to do is just to go out and doesn't have doing higher energy case. But the point is it's not going on the straight loss. But when you look at it, you see a straight line. Right? That's the fundamental mystery one that we that how we describe the thing when we're not looking at it is different than what we see when we look at. So when you're in suit of an understanding your deeper understanding of of quantum mechanics."},{"start":1166.4712,"end":1172.1323,"utterance":"What you... When you you're thinking about people from the nineteen hundreds that are. Sort of basically"},{"start":1173.1501,"end":1187.4451,"utterance":"getting that going to understand this stuff. When when you're talking about this lack of funding and the lack of encouragement for people to pursue quantum mechanics, you strongly feel like there are answers to..."}]

In [ ]:
# meeting_df = pd.read_json("transcript.json")
meeting_df = pd.DataFrame(transcript)
meeting_df = meeting_df.sort_values("start")
# for i in range(1):
#   copied_meeting = meeting_df.copy()
#   copied_meeting["start"]+= meeting_df["end"].max()
#   copied_meeting["end"]+= meeting_df["end"].max()
#   meeting_df = pd.concat([meeting_df, copied_meeting]).reset_index(drop=True)
meeting_df["utterance"] = meeting_df.rename({"utterance": "turn"}, axis=1)["turn"].map(sent_tokenize)
# meeting_df = meeting_df.drop(["turn"], axis=1)
meeting_df = meeting_df.explode("utterance").reset_index(drop=True)
meeting_duration = meeting_df["end"].max() - meeting_df["start"].min()


meeting_duration/60, len(meeting_df)

(19.719964666666666, 291)

In [ ]:
segments = segment_meeting(
    meeting_df, utterance_col="utterance",
    batch_size=10, 
    sentence_comparision_window=5,
    num_passes=2,
    gap_window=2,
    capped=True,
    average_segment_length=50, 
    max_segments=10,
    threshold_fraction=0.7)

281
279


In [ ]:
 def extract_segment_highlights(docs):
  doc = Doc.from_docs(docs)
  
  candidates = [sentence.text for sentence in doc.sents if len(sentence) > 3]
  
  candidates = kw_model.extract_keywords(
      doc.text, candidates=candidates, use_mmr=True, diversity=0.3, top_n=25)
  candidates, _ = zip(*candidates)
  candidates = list(candidates)
  candidates = list(nlp.pipe(candidates))
  
  candidate_sentence_topics = extract_candidate_topics(candidates)
  candidate_sentence_topics, _ = zip(*candidate_sentence_topics)
  
  candidate_topics = extract_candidate_topics(docs)
  candidate_topics, _ = zip(*candidate_topics)
  
  candidates_list = list(candidate_sentence_topics) + list(candidate_topics)

  candidates = kw_model.extract_keywords(
      doc.text, candidates=candidates_list,
      stop_words="english", use_mmr=True, diversity=0.9, top_n=5)
  return candidate_topics


In [ ]:
if segments:
  for segment in segments:
    print(f"segment_length: {len(segment)}")
    print(f"start: {segment['start'].min():2.2f},\t"
        f"end: {segment['end'].max():2.2f}, \t"
        f"duration: {segment['end'].max() - segment['start'].min():2.2f}")
    segment_text = list(nlp.pipe(segment["utterance"].tolist()))
    segment_topics = extract_segment_highlights(segment_text)
    pprint(segment_topics)
    display(segment.head(5))
    display(segment.tail(5))
    print()

segment_length: 65
start: 4.25,	end: 221.41, 	duration: 217.16
('some quantum mechanics', 'make custom', 'score', 'google opening', 'spending')


,start,end,utterance
0,4.24722,68.6,"You've done an amazing job in this book, so tr..."
1,4.24722,68.6,Click hard it is it is a complicated and insan...
2,4.24722,68.6,Yeah.
3,4.24722,68.6,And it's it's one of those things where it lik...
4,4.24722,68.6,The the the just the possibility that there's ...


,start,end,utterance
60,184.61758,221.41086,You only ever get one or two answers it's eith...
61,184.61758,221.41086,I just recording a meeting.
62,184.61758,221.41086,Hey.
63,184.61758,221.41086,Hi.
64,184.61758,221.41086,Can you hear me?



segment_length: 153
start: 224.38,	end: 928.40, 	duration: 704.02
('minutes lengthy',
 'need connect',
 'clothing thing',
 'the discussion initial level discussion',
 'let scroll')


,start,end,utterance
65,224.37755,233.71277,Yeah.
66,224.37755,233.71277,I was Yeah.
67,224.37755,233.71277,Was recording a meeting.
68,224.37755,233.71277,"Podcast cars, I was running it."
69,224.37755,233.71277,"So three minutes fifty two seconds, I think te..."


,start,end,utterance
213,914.265,919.185,I'm just pushing with no verified flags.
214,920.385,922.485,Be before I.
215,925.040,928.400,Hello?
216,925.040,928.400,Yeah.
217,925.040,928.400,Cool.



segment_length: 73
start: 925.04,	end: 1187.45, 	duration: 262.41
('spin measured',
 'send deal',
 'oriented vertically',
 'Just those two possibilities',
 'another magnet')


,start,end,utterance
218,925.0400,928.4000,Okay.
219,930.3333,930.8914,Okay.
220,930.3333,930.8914,Bye.
221,939.6400,940.1400,Okay.
222,941.3200,968.4192,So part the quantum mechanics that that's the ...


,start,end,utterance
286,1166.4712,1172.1323,What you...
287,1166.4712,1172.1323,When you you're thinking about people from the...
288,1166.4712,1172.1323,Sort of basically
289,1173.1501,1187.4451,getting that going to understand this stuff.
290,1173.1501,1187.4451,When when you're talking about this lack of fu...
